In [36]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import boto3
import time
import datetime
import math
import decimal
import json
from boto3.dynamodb.conditions import Key, Attr
from utilities import *

In [37]:
# accessing Amazon Dynamodb
dynamodb = boto3.resource('dynamodb')

# getting access to DataSummary table
vehicle_data_table = dynamodb.Table('vehicle_data')

print(vehicle_data_table.creation_date_time) # test if connected to the database

2022-03-24 18:08:20.240000-05:00


In [38]:
raw_data = pd.read_csv("../data/test1.csv")

In [39]:
type(raw_data['trip'][1].item())
# localtime = time.localtime(time.time())
# localtime

int

In [40]:
for i in range(raw_data.trip.size):
    vehicle_data_table.put_item(
        ##### Change the table name here if necessary #####
        TableName = "vehicle_data",
        Item = {
            "trip": raw_data.trip[i].item(),
            "tm": raw_data.tm[i].item(),
            'rpm': raw_data.rpm[i].item(),
            'speed': raw_data.speed[i].item(),
            'throttle_pos': raw_data.throttle_pos[i].item(),
            'acc_x': raw_data.acc_x[i].item(),
            'acc_y': raw_data.acc_y[i].item(),
            'acc_z': raw_data.acc_z[i].item(),
            'gyro_x': raw_data.gyro_x[i].item(),
            'gyro_y': raw_data.gyro_y[i].item(),
            'gyro_z': raw_data.gyro_z[i].item(),
        }
    )